# Конвейер

## Предобработка с помощью токенизатора

In [69]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
inputs = tokenizer(['Wasup, my nigger', 'Hello, my name is Ivan the Berg'], return_tensors='pt', padding=True,
                   truncation=True)
inputs

{'input_ids': tensor([[  101,  2001,  6279,  1010,  2026,  9152, 13327,   102,     0,     0],
        [  101,  7592,  1010,  2026,  2171,  2003,  7332,  1996, 15214,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## Проходя сквозь модель

**AutoModel** - загружает только основную нейросетевую архитектуру (BERT, ...), но без специфической "головы" (выходного слоя).

- Подходит для задач, где нужна эмбеддинговая (скрытая) информация.

- Требует добавления выходного слоя вручную при обучении.


**AutoModel** вернет только обработанные механизмом внимания эмбеддинги, поэтому используются **AutoModelFor..**, которые и подставляют соответсвующую голову

In [70]:
from transformers import AutoModel

model = AutoModel.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 10, 768])


In [71]:
outputs

BaseModelOutput(last_hidden_state=tensor([[[-0.2583,  0.5758, -0.1630,  ..., -0.1796,  0.1406,  0.1123],
         [-0.3025,  0.4144,  0.0608,  ...,  0.2308,  0.0462,  0.3716],
         [-0.1415,  0.6211, -0.0550,  ...,  0.0570, -0.0231, -0.3812],
         ...,
         [ 0.1551,  0.2540,  0.0804,  ..., -0.0770, -0.1896, -0.2284],
         [-0.0036,  0.5429,  0.3276,  ..., -0.1484,  0.1733,  0.0077],
         [-0.1369,  0.5941,  0.0980,  ..., -0.0386,  0.1710,  0.1028]],

        [[ 0.1183,  0.5219,  0.3419,  ...,  0.8541,  1.0013,  0.0197],
         [ 0.4419,  0.6374,  0.3281,  ...,  0.6219,  1.0461, -0.1147],
         [-0.1589,  0.9009,  0.2998,  ...,  0.4126,  0.8258,  0.0569],
         ...,
         [ 0.2550,  0.1827,  0.5216,  ...,  1.0659,  0.8330,  0.6071],
         [-0.1991,  0.0560,  1.0398,  ...,  0.8049,  0.6226, -0.0513],
         [ 0.8422,  0.2528,  0.6130,  ...,  0.8830,  0.1598, -0.3166]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)

## Головы моделей: Извлечение смысла из чисел

In [72]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
outputs = model(**inputs)
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.6755, -1.4697],
        [-2.9174,  3.1364]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

## Постобработка вывода

In [73]:
from torch import argmax, softmax

softmax_predictions = softmax(outputs.logits, dim=-1)
argmax_predictions = argmax(outputs.logits, dim=-1)
softmax_predictions, argmax_predictions

(tensor([[0.9587, 0.0413],
         [0.0023, 0.9977]], grad_fn=<SoftmaxBackward0>),
 tensor([0, 1]))

Также можно получить лейблы модели

In [74]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

# Модели

## AutoModel

Скачает модель с весами, но без головы, те возвращать данная модель будет только обработанный механизмом внимания изначальный эмбеддинг

In [75]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained("bert-base-cased")
print(type(bert_model))

gpt_model = AutoModel.from_pretrained("gpt2")
print(type(gpt_model))

bart_model = AutoModel.from_pretrained("facebook/bart-base")
print(type(bart_model))

<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
<class 'transformers.models.bart.modeling_bart.BartModel'>


## Config

Также можно скачать саму структуру модели без весов

Можно скачать конфиг (структуру) модели без весов

In [76]:
from transformers import AutoConfig

bert_config = AutoConfig.from_pretrained("bert-base-cased",
                                         num_hidden_layers=3)  # Можно менять архитектуру модели, данное изменение ускорит работу, но ухудшит качество
print(type(bert_config))

gpt_config = AutoConfig.from_pretrained("gpt2")
print(type(gpt_config))

bart_config = AutoConfig.from_pretrained("facebook/bart-base")
print(type(bart_config))

<class 'transformers.models.bert.configuration_bert.BertConfig'>
<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>
<class 'transformers.models.bart.configuration_bart.BartConfig'>


In [77]:
bert_config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.50.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

В данные конфиги можно загрузить веса модели

In [78]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained("bert-base-cased", config=bert_config)
print(type(bert_model))

gpt_model = AutoModel.from_pretrained("gpt2", config=gpt_config)
print(type(gpt_model))

bart_model = AutoModel.from_pretrained("facebook/bart-base", config=bart_config)
print(type(bart_model))

<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
<class 'transformers.models.bart.modeling_bart.BartModel'>


Или можно использовать уже подготовленный **ModelConfig**

In [79]:
from transformers import BertConfig, GPT2Config, BartConfig

bert_config = BertConfig.from_pretrained("bert-base-cased")
print(type(bert_config))

gpt_config = GPT2Config.from_pretrained("gpt2")
print(type(gpt_config))

bart_config = BartConfig.from_pretrained("facebook/bart-base")
print(type(bart_config))

<class 'transformers.models.bert.configuration_bert.BertConfig'>
<class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>
<class 'transformers.models.bart.configuration_bart.BartConfig'>


Данные конфиги также можно загрузить в "оболочку" модели

In [80]:
from transformers import BertModel, GPT2Model, BartModel

bert_model = BertModel.from_pretrained("bert-base-cased")
print(type(bert_model))

gpt_model = GPT2Model.from_pretrained("gpt2")
print(type(gpt_model))

bart_model = BartModel.from_pretrained("facebook/bart-base")
print(type(bart_model))

<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.gpt2.modeling_gpt2.GPT2Model'>
<class 'transformers.models.bart.modeling_bart.BartModel'>


## Saving and reloading

In [81]:
from transformers import AutoModel

bert_model = AutoModel.from_pretrained("bert-base-cased")
bert_model.save_pretrained("my-bert-model")

bert_model = AutoModel.from_pretrained("my-bert-model")
type(bert_model)

transformers.models.bert.modeling_bert.BertModel

## Inference

In [82]:
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model(**bert_tokenizer('zaza boss is me', return_tensors='pt')).last_hidden_state

tensor([[[ 0.4535,  0.3524,  0.1490,  ..., -0.2225,  0.3883, -0.1498],
         [ 0.2127, -0.1671,  0.3893,  ...,  0.4731, -0.5549, -0.0263],
         [ 0.5378,  0.3529,  0.4334,  ...,  0.0068,  0.0505,  0.0287],
         ...,
         [ 0.4239,  0.0934,  0.1885,  ..., -0.1535,  0.0420, -0.1724],
         [ 0.4564, -0.1387,  0.2731,  ..., -0.4077,  0.1747, -0.1223],
         [ 1.3320,  0.1104,  0.0837,  ..., -0.1996,  1.2236, -0.5436]]],
       grad_fn=<NativeLayerNormBackward0>)

# Токенизаторы

У некоторых токенизаторов есть дополнительно поле **token_type_ids**

In [83]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
inputs = tokenizer('Wasup, my nigger', 'Hello, my name is Ivan the Berg', return_tensors='pt', padding=True,
                   truncation=True)
inputs

{'input_ids': tensor([[  101,  3982,  4455,   117,  1139, 11437,  9146,   102,  8667,   117,
          1139,  1271,  1110,  7062,  1103, 16218,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Оно обозначает принадлежность того или иного токена к каждому из переданных предложений. Первое предложение - 0, второе - 1 и так далее

In [84]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
inputs = tokenizer('Wasup, my nigger', 'Hello, my name is Ivan the Berg', return_tensors='pt', padding=True,
                   truncation=True)
inputs

{'input_ids': tensor([[  101,  3982,  4455,   117,  1139, 11437,  9146,   102,  8667,   117,
          1139,  1271,  1110,  7062,  1103, 16218,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

## Encoding

Перевод текста в числа называется кодированием (encoding). Кодирование выполняется в два этапа: токенизация, а затем преобразование во входные идентификаторы.

Разбиение текста на токены

In [85]:
from transformers import AutoTokenizer

bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert_tokenizer.tokenize("Let's try to tokenize!")

['Let', "'", 's', 'try', 'to', 'token', '##ize', '!']

In [86]:
from transformers import AutoTokenizer

albert_tokenizer = AutoTokenizer.from_pretrained('albert-base-v1')
albert_tokenizer.tokenize("Let's try to tokenize!")

['▁let', "'", 's', '▁try', '▁to', '▁to', 'ken', 'ize', '!']

Переводим каждый токен в **ID**

In [87]:
bert_token_ids = bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize("Let's try to tokenize!"))
bert_token_ids

[2421, 112, 188, 2222, 1106, 22559, 3708, 106]

In [88]:
albert_token_ids = albert_tokenizer.convert_tokens_to_ids(albert_tokenizer.tokenize("Let's try to tokenize!"))
albert_token_ids

[408, 22, 18, 1131, 20, 20, 2853, 2952, 187]

Но таким методом мы не учитываем особые токены вида EOS, применим иной метод для этого

In [89]:
bert_tokenizer.prepare_for_model(bert_token_ids)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': [101, 2421, 112, 188, 2222, 1106, 22559, 3708, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [90]:
albert_tokenizer.prepare_for_model(albert_token_ids)

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': [2, 408, 22, 18, 1131, 20, 20, 2853, 2952, 187, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Посмотрим на обратный метод, что из id сделает токен

In [91]:
bert_tokenizer.decode(bert_tokenizer.prepare_for_model(bert_token_ids)['input_ids'])

"[CLS] Let ' s try to tokenize! [SEP]"

In [92]:
albert_tokenizer.decode(albert_tokenizer.prepare_for_model(albert_token_ids)['input_ids'])

"[CLS] let's try to tokenize![SEP]"

## Handling multiple sequences

In [93]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor(ids)
print("Input IDs:", input_ids)

# Эта строка выдаст ошибку.
model(input_ids)

Input IDs: tensor([ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
         2026,  2878,  2166,  1012])


IndexError: too many indices for tensor of dimension 1

Ошибка сверху вызвана тем, что размерности 1 нам недостаточно

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Добавив ещё одну размерность ошибка была исправлена

**Батчинг** - это отправка нескольких предложений через модель одновременно.

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200]  # выдаст ошибку, тк размеры всех энкодингов должны быть одинаковыми
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

Для решения данный проблемы использется **Padding** - добавление пустных значений для достижения одинаковых размеров токенизированных строк

In [ ]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
print('Pad Token ID - ', tokenizer.pad_token_id)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id]
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

Что-то не так с логитами в наших батчах: во втором ряду должны быть те же логиты, что и для второго предложения, но мы получили совершенно другие значения!

Это связано с тем, что ключевой особенностью моделей Transformer являются слои внимания (attention layers), которые контекстуализируют каждый токен. Они учитывают токены дополнений, так как рассматривают все токены последовательности. Чтобы получить одинаковый результат при прохождении через модель отдельных предложений разной длины или при прохождении батча с одинаковыми предложениями и дополнениями, нам нужно указать слоям внимания игнорировать дополняющие токены. Для этого используется маска внимания (attention mask).

In [ ]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(outputs.logits)

Всё это реализовано под капотом библиотеки **transformers**

In [94]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer(['Hello, my name is Ivan the Berg', 'I am a nigger'], padding=True)

{'input_ids': [[101, 7592, 1010, 2026, 2171, 2003, 7332, 1996, 15214, 102], [101, 1045, 2572, 1037, 9152, 13327, 102, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}

# Собираём всё воедино

In [96]:
from transformers import AutoModelForSequenceClassification
from torch import softmax

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

texts = ['Hello, my name is Ivan the Berg', 'I am a nigger']
tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
probs = softmax(output.logits, dim=-1)

for text, prob in zip(texts, probs):
    result = {model.config.id2label[i]: float(prob[i]) for i in range(len(prob))}
    print(f"Text: {text}")
    print("Predictions:", result, '\n')

Text: Hello, my name is Ivan the Berg
Predictions: {'NEGATIVE': 0.0023433719761669636, 'POSITIVE': 0.9976565837860107} 

Text: I am a nigger
Predictions: {'NEGATIVE': 0.9906907081604004, 'POSITIVE': 0.009309322573244572} 

